일시 : 2020-07-08

팀원 : 박성찬, 강민지, 천희국

내용 : python과 SQL을 이용하여 회원가입/로그인 서비스 만들기 

---

## [1] 회원가입 만들기(csv 사용)

1. 로그인/회원가입을 처음에 인풋으로 분기
2. 인풋으로 아이디와 비밀번호를 받고, 이것을 토대로 csv 파일에다가 저장한다.

3. 로그인할 때는 csv 파일을 검사해서, 로그인 성공인지 실패인지 여부를 띄운다.

In [5]:
import csv

check = input("아이디가 있습니까?(Y/N)").upper()



if check == 'Y' : # -- 로그인(희국)
  print('----------로그인----------')


  ## ??? id가 있는지 확인 
  # 1) 중간에 id가 있으면 패스워드 입력 -- done 
  # 2) 다 봤는데도 id가 없으면 print('아이디가 존재하지 않습니다.)
  flag = 1
  while flag:
    id = input('아이디를 입력하세요: ')  
    
    with open('a.csv', 'r') as csvfile:
      csvreader = csv.reader(csvfile)

      user_check = 0
      for i in csvreader:
        if id == i[2]:
          flag = 0
          user_check = 1
          break 
      
      if user_check == 0:
        print("아이디가 존재하지 않습니다.")
      
           
        
  
  checker1 = 0
  while checker1 == 0:
    pw = input("비밀번호를 입력하세요: ")

    with open('a.csv', 'r') as csvfile:
      csvreader = csv.reader(csvfile)
      
      for i in csvreader:
        if(id == i[2] and pw == i[3]) :
          print('로그인 성공!')
          checker1 = 1
          break
        elif(id == i[2] and pw !=i[3]) :
          print('비밀번호가 틀렸습니다.')


    
elif check == 'N': # -- 회원가입(성찬)
  print('----------회원가입----------')
  with open('a.csv', 'a', newline = '\n') as csvfile:
    csvwriter = csv.writer(csvfile)
    id = ''
    password = ''

    name = input("이름을 입력해주세요: ")
    YMD = input("생년월일을 입력해주세요(0000-00-00): ")
    while not id:
      id = input("아이디를 입력해주세요: ")
    ## 중복되는 id가 있는지
    with open('a.csv', 'r') as csvfile:
      csvreader = csv.reader(csvfile)
      for i in csvreader:
        check = 0
        while id == i[2] and check == 0:
          id = input("이미 있는 아이디입니다. 다른 아이디를 입력해주세요: ")
          if(id != i[2]) :
            check = 1
            break
          

    while password == '':
      password = input('비밀번호를 입력해주세요.')             

    csvwriter.writerow([name, YMD, id, password])
    print('----------가입완료----------')

  

else:
  print("잘못 입력하셨습니다. 다시 수행해주세요.")

아이디가 있습니까?(Y/N)y
----------로그인----------
아이디를 입력하세요: id3
아이디가 존재하지 않습니다.
아이디를 입력하세요: id1
비밀번호를 입력하세요: p
비밀번호가 틀렸습니다.
비밀번호를 입력하세요: pw1
로그인 성공!


---

## [2] SQL을 통한 로그인 구현

1. 로그인/회원가입을 인풋으로 분기

2. 인풋으로 아이디와 비번을 받고, 이것을 토대로 db에 쿼리를 날려서 저장

3. 로그인할때는 db 파일을 읽어와서, 로그인 성공인지 실패인지 여부를 띄운다

In [6]:
import sqlite3 
conn = sqlite3.connect('USER.db')
cur = conn.cursor()

cur.execute("""CREATE TABLE USER(
        name TEXT,
        birth TEXT,
        phone TEXT,
        gender TEXT,
        id varchar(10) primary key,
        password varchar(15) not null)""")

In [10]:
def signup(id, password, name='', birth='', phone='', gender=''): # 회원가입
  cur = conn.cursor()
  sql = "INSERT INTO USER VALUES (?, ?, ?, ?, ?, ?)"
  new_user = [name, birth, phone, gender, id, password]
  cur.execute(sql, new_user)
  conn.commit()
  conn.close()
  print("가입이 완료되었습니다.")
  return 0


def signin(id, password): # 로그인
  cur = conn.cursor()
  sql = "SELECT id, password FROM USER WHERE id = ? and password = ?"
  x = [id, password]
  cur.execute(sql, x)
  result = cur.fetchall()
  if len(result) == 1:
    print("로그인 성공!")
  else:
    print("로그인 실패ㅜㅜ 다시 수행해주세요.")
  

In [11]:
# 로그인/회원가입 선택
check = input("아이디가 있습니까?(Y/N)").upper()
conn = sqlite3.connect('USER.db')

# 회원가입 -> id, password input으로 받아서 db에 쿼리 날려 저장
if check == 'N' :
  flag = 1
  while flag:
    print('----------회원가입----------')
    id = input("아이디를 입력하세요(필수): ")
    pw = input("비밀번호를 입력하세요(필수): ")
    name = input("이름을 입력하세요: ")
    birth = input("생년월일을 입력하세요(0000-00-00): ")
    phone = input("전화번호를 입력하세요(000-0000-0000): ")
    gender = input('성별을 입력하세요(남/여): ')
    
    try:
      flag = signup(id, password, name, birth, phone, gender)
    except:
      print('아이디가 이미 존재합니다. 다시 수행해주세요.')

# 로그인 -> db파일 읽어와서 로그인 성공, 실패 여부 띄우기
elif check == 'Y' :
  print('----------로그인----------')
  id = input('아이디를 입력하세요: ')  
  pw = input("비밀번호를 입력하세요: ")


else:
  print("잘못 입력하셨습니다. 다시 수행해주세요.")

아이디가 있습니까?(Y/N)n
----------회원가입----------
아이디를 입력하세요(필수): id5
비밀번호를 입력하세요(필수): pw5
이름을 입력하세요: name5
생년월일을 입력하세요(0000-00-00): 5555-55-5
전화번호를 입력하세요(000-0000-0000): 555-5555-5555
성별을 입력하세요(남/여): 여
아이디가 이미 존재합니다. 다시 수행해주세요.
----------회원가입----------
아이디를 입력하세요(필수): id3
비밀번호를 입력하세요(필수): pw3
이름을 입력하세요: name3
생년월일을 입력하세요(0000-00-00): 3333-33-33
전화번호를 입력하세요(000-0000-0000): 333-3333-3333
성별을 입력하세요(남/여): 여
가입이 완료되었습니다.
